In [1]:
import numpy as np
import cv2
import time
import concurrent.futures as multithreading

In [3]:
class Video:
    
    @staticmethod
    def compress(filename, fraction=0.01):

        def get_indices(height, width, fraction):

            def to_2d(index):
                return (index // width, index % width)

            samples = np.random.choice(
                        np.arange(0, width * height), 
                        size = int(width * height * fraction), 
                        replace=False
                      )

            indices = [to_2d(index) for index in samples]

            return tuple(zip(*indices))


        def sample_frame(frame, fraction):
            height, width, nb_channels = frame.shape

            indices = get_indices(height, width, fraction)

            return np.expand_dims(frame[indices], axis=0), np.expand_dims(indices, axis=0)


        cap = cv2.VideoCapture(filename)
        framerate = np.array(cap.get(cv2.cv2.CAP_PROP_FPS))

        num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        start_time = time.time()

        ret, frame = cap.read()
        dimensions = np.array(frame.shape)

        video, indices = sample_frame(frame, fraction)
        curr_frame = 1

        while cap.isOpened():
            ret, frame = cap.read()

            if not ret:
                break

            sampled_frame, sampled_indices = sample_frame(frame, fraction)

            video = np.vstack((video, sampled_frame))
            indices = np.vstack((indices, sampled_indices))

            curr_frame += 1
            print('Compressing Video: ' + str(int(100 * curr_frame / num_frames)) + 
                  '%\tTime Elapsed: ' + str(int(np.floor(time.time() - start_time))) + ' seconds', end='\r')

        cap.release()

        print('Compressing Video: 100%\tTime Elapsed: ' + str(int(np.floor(time.time() - start_time))) + ' seconds')

        np.savez_compressed(
            filename.split('.')[0] + '_compressed', 
            video=video, 
            indices=indices, 
            dimensions=dimensions, 
            framerate=framerate
        )

In [4]:
class Kernels:
    
    @staticmethod
    def kernel_2d(kernel_size, sigma):
        center = kernel_size // 2
        kernel = np.fromfunction(
            lambda x, y: 
                np.exp( -0.5 * ((x - center) ** 2 + (y - center) ** 2) / (sigma ** 2)), 
            (kernel_size, kernel_size), 
            dtype=float)

        return kernel
    
    @staticmethod
    def kernel_3d(kernel_size, sigma, num_kernels, sigma_time):
        center = kernel_size // 2
        kernels = np.fromfunction(
            lambda t, x, y: 
                np.exp( -0.5 * (
                                 (((x - center) ** 2 + (y - center) ** 2) / (sigma ** 2)) + 
                                  ((t ** 2) / (sigma_time ** 2))
                               )
                      ), 
            (num_kernels, kernel_size, kernel_size), 
            dtype=float
        )
        return kernels


In [5]:
class CompressedVideo:
    
    def __init__(self, filename):
        
        file_dict = np.load(filename)
        
        self.filename = filename.split('_compressed.npz')[0]
        self.height = file_dict['dimensions'][0]
        self.width = file_dict['dimensions'][1]
        self.nb_channels = file_dict['dimensions'][2]
        self.video = file_dict['video']
        self.indices = file_dict['indices']
        self.num_frames = self.video.shape[0]
        self.framerate = file_dict['framerate']
        
        self.defaults = {'kernel_size':     71, 
                         'sigma':           7, 
                         'num_time_frames': 11, 
                         'output_dst':      self.filename + '_reconstructed.mov',
                         'multithreaded':   True}
    
    
    def reconstruct(self, algorithm='efan2d', **kwargs):
        
        if 'kernel_size' in kwargs:
            kernel_size = kwargs['kernel_size']
        else:
            kernel_size = self.defaults['kernel_size']
        
        if 'sigma' in kwargs:
            sigma = kwargs['sigma']
        else:
            sigma = self.defaults['sigma']
        
        if 'num_time_frames' in kwargs:
            num_time_frames = kwargs['num_time_frames']
        else:
            num_time_frames = self.defaults['num_time_frames']
        
        if 'output_dst' in kwargs:
            output_dst = kwargs['output_dst']
        else:
            output_dst = self.defaults['output_dst']
        
        if 'multithreaded' in kwargs:
            multithreaded = kwargs['multithreaded']
        else:
            multithreaded = self.defaults['multithreaded']
        
        
        def display_progress(frame_index, start_time, filter_times, complete=False):
            print('Reconstructing Video: ' + str(int(100 * frame_index / self.num_frames)) + '% ' + 
                  '\tTime Elapsed: ' + str(int(np.floor(time.time() - start_time))) + ' seconds ' +
                  '\tFilter Time: '  + str(int(np.floor(np.sum(filter_times)))) + ' seconds', end='\r')
            
            if complete:
                print('')
        
        
        def prepare_frame(frame_index):
            frame = self.video[frame_index]
            indices = self.indices[frame_index]

            black_frame = np.zeros((self.height, self.width, self.nb_channels), dtype=np.uint8)
            black_frame[tuple(indices)] = frame

            new_channel = np.zeros((self.height, self.width, 1), dtype=np.uint8)
            new_channel[tuple(indices)] = 1

            augmented_frame = np.append(black_frame, new_channel, axis=2).astype(float)
            
            return augmented_frame
        
        
        def normalize_frame(frame):
            for i in range(self.nb_channels):
                frame[:,:,i] /= frame[:,:,-1]

            reconstructed_frame = frame[:,:,:-1].astype(np.uint8)
            
            return reconstructed_frame
        
        
        def efan2d(output):
            """
            EFAN2D Algorithm
            """
            
            filter_times = []

            def filter_frame(frame_index, kernel):
                augmented_frame = prepare_frame(frame_index)

                st = time.time()
                filtered = cv2.filter2D(augmented_frame, -1, kernel)
                filter_times.append(time.time() - st)
                
                return filtered
            
            
            def reconstruct_frame(frame_index, kernel):
                filtered = filter_frame(frame_index, kernel)
                
                reconstructed_frame = normalize_frame(filtered)
                # Not sure about this, paper formula for reconstruction does not include it
                #reconstructed_frame[tuple(indices)] = frame
                
                return reconstructed_frame
            
            
            start_time = time.time()
            
            kernel = Kernels.kernel_2d(kernel_size, sigma)
            
            display_progress(0, start_time, filter_times)
            
            for frame_index in range(self.num_frames):
                output.write(reconstruct_frame(frame_index, kernel))
                display_progress(frame_index, start_time, filter_times)
            
            display_progress(self.num_frames, start_time, filter_times, complete=True)


        def efan3d(output):
            """
            EFAN3D Algorithm
            """
            
            num_kernels = (num_time_frames + 1) // 2
            sigma_time = (num_kernels - 1) / 6
            
            filter_times = []
            
            def filter_frame(frame_index, kernels):
                augmented_frame = prepare_frame(frame_index)

                st = time.time()
                
                if multithreaded:
                    filtered_frame = [None for i in range(num_kernels)]

                    def apply_kernel(kernel_index):
                        filtered = cv2.filter2D(augmented_frame, -1, kernels[kernel_index])
                        filtered_frame[kernel_index] = filtered

                    with multithreading.ThreadPoolExecutor(max_workers=num_kernels) as executor:
                        executor.map(apply_kernel, range(num_kernels))
                
                else:
                    filtered_frame = []
                    for kernel in kernels:
                        filtered = cv2.filter2D(augmented_frame, -1, kernel)
                        filtered_frame.append(filtered)
                
                filter_times.append(time.time() - st)
                
                return filtered_frame
            
            
            def reconstruct_frame(frame_index, filtered_frames):
                filtered_sum = np.zeros((self.height, self.width, self.nb_channels + 1))
            
                nb_past_frames = min(frame_index, num_kernels - 1)
                for idx in range(nb_past_frames):
                    filtered_sum += filtered_frames[idx][nb_past_frames - idx]

                nb_future_frames = min(self.num_frames - frame_index, num_kernels)
                for idx in range(nb_future_frames):
                    filtered_sum += filtered_frames[min(frame_index, num_kernels - 1) + idx][idx]
                
                reconstructed_frame = normalize_frame(filtered_sum)
                
                return reconstructed_frame
            
            
            start_time = time.time()
            
            kernels = Kernels.kernel_3d(kernel_size, sigma, num_kernels, sigma_time)
            
            filtered_frames = []
            
            display_progress(0, start_time, filter_times)
            
            for frame_index in range(self.num_frames):
                
                if frame_index == 0:
                    for idx in range(num_kernels):
                        filtered_frames.append(filter_frame(idx, kernels))
                        
                elif frame_index < num_kernels:
                    filtered_frames.append(filter_frame((num_kernels - 1) + frame_index, kernels))
                    
                elif frame_index <= self.num_frames - num_kernels:
                    filtered_frames.append(filter_frame((num_kernels - 1) + frame_index, kernels))
                    filtered_frames = filtered_frames[1:]
                    
                else:
                    filtered_frames = filtered_frames[1:]
                
                output.write(reconstruct_frame(frame_index, filtered_frames))
                
                display_progress(frame_index, start_time, filter_times)
            
            display_progress(self.num_frames, start_time, filter_times)
        
        
        def adefan(output):
            pass
            
        
        output = cv2.VideoWriter(
                    output_dst, 
                    cv2.VideoWriter_fourcc('M','J','P','G'), 
                    self.framerate, 
                    (self.width, self.height)
                 )
        
        if algorithm == 'efan2d':
            efan2d(output)
        elif algorithm == 'efan3d':
            efan3d(output)
        elif algorithm == 'adefan':
            adefan(output)
        
        output.release()
    

In [15]:
Video.compress('Beach.mp4')

Compressing Video: 100%	Time Elapsed: 42 seconds


In [16]:
video = CompressedVideo('Beach_compressed.npz')

In [17]:
video.reconstruct(algorithm='efan2d', output_dst='ef2.mov')

Reconstructing Video: 100% 	Time Elapsed: 129 seconds 	Filter Time: 92 seconds


In [18]:
video.reconstruct(algorithm='efan3d', multithreaded=True, output_dst='ef3.mov')

In [19]:
video.reconstruct(algorithm='efan3d', multithreaded=False, output_dst='ef3singlethreaded.mov')

In [20]:
Video.compress('Seal.mp4')

Compressing Video: 100%	Time Elapsed: 1 seconds


In [21]:
seal = CompressedVideo('Seal_compressed.npz')

In [22]:
seal.reconstruct(algorithm='efan2d', output_dst='seal_ef2.mov')

Reconstructing Video: 100% 	Time Elapsed: 8 seconds 	Filter Time: 6 seconds


In [23]:
seal.reconstruct(algorithm='efan3d', multithreaded=True, output_dst='seal_ef3.mov')

In [24]:
seal.reconstruct(algorithm='efan3d', multithreaded=False, output_dst='seal_ef3singlethreaded.mov')

In [29]:
##################################
##### Optimization Benchmark #####
##################################

height = 720
width = 1280
num_channels = 3
length = 38 #seconds
framerate = 24
num_frames = length * framerate
anchor_x = 400
anchor_y = 400
num_rand_pixels = (height * width) // 100

kernel_size = 71
kernel = Kernels.kernel_2d(kernel_size, 20)
r = (kernel_size - 1) // 2

###########################
##### Naive Operation #####
###########################

st = time.time()

kernel_dict = {}
for i in range(256):
    kernel_dict[i] = kernel * i

filtered = np.zeros((height, width, num_channels + 1));

for idx in range(num_rand_pixels // 100):
    k = kernel_dict[np.random.randint(0,256)]
    for c in range(num_channels + 1):
        for y in range(kernel_size):
            for x in range(kernel_size):
                filtered[anchor_x-r+y,anchor_y-r+x,c] += k[x,y]
tot_time_naive = (time.time() - st) * num_frames * 100
print('Naive python operations / array indexing:', np.around(tot_time_naive / 3600, 1), 'hours')

###########################
##### Numpy Optimized #####
###########################

st = time.time()

filtered = filtered = np.zeros((height, width, num_channels + 1));

kernel_dict = {}
for i in range(256):
    kernel_dict[i] = kernel * i

for i in range(num_rand_pixels):
    for c in range(num_channels + 1):
        filtered[anchor_y-r:anchor_y+r+1, anchor_x-r:anchor_x+r+1, c] += kernel_dict[np.random.randint(0,256)]

tot_time_numpy = (time.time() - st) * num_frames
print('Numpy (C) operations / array indexing:', np.around(tot_time_numpy / 60, 1), 'minutes')
print('Numpy speedup:', np.around(tot_time_naive / tot_time_numpy, 1), 'times faster')

Naive python operations / array indexing: 29.6 hours
Numpy (C) operations / array indexing: 9.6 minutes
Numpy speedup: 185.0 times faster


The equivalent of the MATLAB / C++ implementation would take over 29 hours to reconstruct the video using python operations and almost 10 minutes using numpy operations. Since numpy is already written in C there isn't much more to do to speed up the additions / multiplications / array indexing. Even using numpy this is still more than 4 times slower than using opencv to interpolate the value of each pixel (Probably because of the time gained by using filter2D which uses the FFT algorithm).